## 3. <a href='https://cwiki.apache.org/confluence/display/Hive/LanguageManual+DDL#LanguageManualDDL-CreateTableCreate/Drop/TruncateTable'>创建表</a>,<a href='https://cwiki.apache.org/confluence/display/Hive/LanguageManual+DDL#LanguageManualDDL-CreateTableCreate/Drop/TruncateTable'>加载数据</a>


#### 创建内部表表,创内部表,在删除表结构的时候,表数据也会被删除.
    create table psn(
            id int,
            name string,
            habit array<string>,
            address map<string,string>
        ) 
        ROW FORMAT DELIMITED 
            FIELDS TERMINATED BY ',' 
            COLLECTION ITEMS TERMINATED BY '-'
            MAP KEYS TERMINATED BY ':'  
            LINES TERMINATED BY '\n';
#### 加载文件    
    LOAD DATA [LOCAL] INPATH '/data/mydata'  INTO TABLE psn;
#### 查看描述
    desc formated psn;

#### 创建外部表,创建的时候要加入LOCATION,HIVE只维护表结构,不会动表数据
     CREATE EXTERNAL TABLE page_view(
         viewTime INT, 
         userid BIGINT,
         page_url STRING, 
         referrer_url STRING,
         ip STRING COMMENT 'IP Address of the User',
         country STRING COMMENT 'country of origination'
         )
     COMMENT 'This is the staging page view table'
         ROW FORMAT DELIMITED FIELDS TERMINATED BY '\054'
         STORED AS TEXTFILE
     LOCATION '<hdfs_location>';
     
     
     CREATE EXTERNAL TABLE psn(
         id INT, 
         name STRING,
         habit ARRAY<STRING>,
         address MAP<STRING,STRING>, 
     )
     ROW FORMAT DELIMITED 
         FIELDS TERMINATED BY ','
         COLLECTION ITEMS TERMINATED BY '-'
         MAP KEYS TERMINATED BY ':'  
         LINES TERMINATED BY '\n';
     LOCATION '/usr';

#### 加载文件    
    LOAD DATA [LOCAL] INPATH '/data/mydata'  INTO TABLE psn;

## 4.分区

###                       a.创建分区
 #### 创建一个分区
     create table psn(
            id int,
            name string,
            habit array<string>,
            address map<string,string>
        ) 
        PARTITIONED BY (age int)
        ROW FORMAT DELIMITED 
            FIELDS TERMINATED BY ',' 
            COLLECTION ITEMS TERMINATED BY '-'
            MAP KEYS TERMINATED BY ':'  
            LINES TERMINATED BY '\n';
 #### 创建两个个分区
      create table psn(
            id int,
            name string,
            habit array<string>,
            address map<string,string>
        ) 
        PARTITIONED BY (age INT),(set,STRING)
        ROW FORMAT DELIMITED 
            FIELDS TERMINATED BY ',' 
            COLLECTION ITEMS TERMINATED BY '-'
            MAP KEYS TERMINATED BY ':'  
            LINES TERMINATED BY '\n';
#### 加载文件    
    LOAD DATA [LOCAL] INPATH '/data/mydata'  INTO TABLE psn PARTITION (age=20, sex='M');
#### 添加分区<a href='https://cwiki.apache.org/confluence/display/Hive/LanguageManual+DDL#LanguageManualDDL-AddPartitions'>help</a>:注意这个不是添加分区字段,是添加字段分区值
    ALTER TABLE psn add PARTITION (age=10,sex='WM')
#### 删除分区<a href='https://cwiki.apache.org/confluence/display/Hive/LanguageManual+DDL#LanguageManualDDL-DropPartitions'>help</a>
    ALTER TABLE psn DROP PARTITION (sex='WM')
#### 问题:外部表drop后会不会删除数据?

## 5.从其他表导入数据
    FROM psn
        INSERT OVERWRITE TABLE psn1   //插入第一个表
        SELECT id,name,habit
        INSERT OVERWRITE TABLE psn2   //插入第二个表
        SELECT id,name


## 6.Hive Beeline(新的客户端) 
    需要配合server2服务使用,不用hive --service metastore &
    1.在node2上面,启动服务 hiveserver2
    2.beeline -u jdbc:hive2://node2:10000/default -n root
    
    命令以!开头
    (2)、beeline
    beeline> !connect jdbc:hive2://<host>:<port>/<db>;auth=noSasl root 123